In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("clustering")

import os
from utils import get_data_dir, prinT

In [15]:
import pickle
with open(os.path.join(get_data_dir(), "2010s","MAG_paper_df.pkl"), "rb") as f:
    MAG_paper_df = pickle.load(f)

with open(os.path.join(get_data_dir(), "2010s","cluster_df.pkl"), "rb") as f:
    cluster_df = pickle.load(f)


data_dir:  /home/lyuzhuoqi/projects/clustering/data
data_dir:  /home/lyuzhuoqi/projects/clustering/data


In [14]:
MAG_paper_df

,Year,DocType,VenueID
PaperID,,,
2049933365,2010,Journal,80951755
2038148770,2010,Journal,119525064
2373199189,2010,Journal,2764482698
2289331308,2010,Journal,2764425571
2784227654,2010,Journal,77020770
...,...,...,...
3185296615,2021,Journal,8391440
3165199696,2021,Journal,120683614
3212184219,2021,Journal,157451995


In [16]:
cluster_df

,Scopus_label,cluster_label,sim_score,closest_center_distance,x_val,y_val
VID,,,,,,
202381698,Multidisciplinary,22,0.004073,0.445886,-67.928200,15.572327
137773608,Multidisciplinary,22,0.004073,0.590942,-68.405334,-55.633186
125754415,Multidisciplinary,22,0.004073,0.574571,-68.448853,-55.613579
3880285,Multidisciplinary,23,0.004789,0.618842,-68.407288,-55.634430
111155417,Chemistry,23,0.251397,0.220853,-54.506985,-61.217068
...,...,...,...,...,...,...
2764485818,Medicine,21,0.097324,0.268070,-28.969574,34.819569
83454320,Arts and Humanities,25,0.459962,0.034777,78.609909,31.736822
16507453,Arts and Humanities,6,0.084561,0.113656,89.206772,17.625090


In [18]:
# 获取所有 PaperID 和对应的期刊 ID
prinT("Mapping paper ID to venue ID")
paper_to_venue = MAG_paper_df["VenueID"].to_dict()
# 获取期刊 ID 和对应的 Scopus_label
prinT("Mapping venue ID to Scopus label")
venue_to_Scopus_label = cluster_df["Scopus_label"].to_dict()
# 获取期刊 ID 和对应的 scluster_label
prinT("Mapping venue ID to cluster label")
venue_to_cluster_label = cluster_df["cluster_label"].to_dict()
# 过滤出有效的 PaperID 和对应 Scopus_label
prinT("Filtering out invalid papers")
valid_papers = {'Scopus_label': {pid: venue_to_Scopus_label[vid] for pid, vid in paper_to_venue.items() if vid in 
                                 venue_to_Scopus_label},
                'cluster_label': {pid: venue_to_cluster_label[vid] for pid, vid in paper_to_venue.items() if vid in 
                                  venue_to_cluster_label}}

2024-11-20 11:45:18 -- Mapping paper ID to venue ID
2024-11-20 11:45:32 -- Mapping venue ID to Scopus label
2024-11-20 11:45:32 -- Mapping venue ID to cluster label
2024-11-20 11:45:32 -- Filtering out invalid papers


In [18]:
len(valid_papers['Scopus_label']), len(valid_papers['cluster_label'])

(29309324, 29309324)

In [23]:
with open(os.path.join(get_data_dir(), "2010s","valid_papers.pkl"), "wb") as f:
    pickle.dump(valid_papers, f)

data_dir:  /home/lyuzhuoqi/projects/clustering/data


# Load `valid_paper.pkl`

In [3]:
import pickle
with open(os.path.join(get_data_dir(), "2010s", "valid_papers.pkl"), "rb") as f:
    valid_papers = pickle.load(f)

data_dir:  /home/lyuzhuoqi/projects/clustering/data


In [ ]:
import ijson
import json
from tqdm import tqdm
import os

def extract_valid_papers(json_path, valid_papers, output_path):
    """
    从平坦的 JSON 文件（{ "PaperID": "abstract", ... }）中提取指定的 PaperID 和摘要。
    
    Args:
        json_path (str): JSON 文件路径。
        valid_papers (dict): PaperID 和其对应标签的字典。
        output_path (str): 提取后保存的文件路径。
    
    Returns:
        int: 匹配到的 PaperID 数量。
    """
    matched_count = 0

    # 获取文件大小用于进度条
    file_size = os.path.getsize(json_path)

    with open(json_path, 'r', encoding='utf-8') as input_file, open(output_path, 'w', encoding='utf-8') as output_file:
        # 初始化 tqdm 进度条
        with tqdm(total=file_size, desc="Processing JSON", unit="B", unit_scale=True) as pbar:
            parser = ijson.kvitems(input_file, "")  # 解析顶层键值对
            batch = []
            batch_size = 50000  # 增大批量写入规模
            counter = 0  # 控制进度条更新频率

            for paper_id, abstract in parser:
                counter += 1
                if counter % 10000 == 0:  # 每 1000 条更新一次进度条
                    pbar.update(input_file.tell() - pbar.n)

                try:
                    paper_id = int(paper_id)
                    if paper_id in valid_papers['Scopus_label']:
                        result = {
                            "PaperID": paper_id,
                            "abstract": abstract,
                            "Scopus_label": valid_papers['Scopus_label'][paper_id],
                            "cluster_label": valid_papers['cluster_label'][paper_id]
                        }
                        batch.append(result)
                        matched_count += 1
                except ValueError:
                    continue

                if len(batch) >= batch_size:
                    output_file.write("\n".join(json.dumps(item) for item in batch) + "\n")
                    batch = []

            if batch:  # 写入剩余的批量数据
                output_file.write("\n".join(json.dumps(item) for item in batch) + "\n")

    prinT(f"Successfully extracted {matched_count} paper abstracts")
    return matched_count


In [ ]:
large_json_path = os.path.join(get_data_dir(), "MAG_abstracts.json")
test_json_path = os.path.join(get_data_dir(), "MAG_abstracts_test.json")
output_path = os.path.join(get_data_dir(), "2010s", "dataset.json")  # 提取后的保存路径

matched_count = extract_valid_papers(large_json_path, valid_papers, output_path)

data_dir:  /home/lyuzhuoqi/projects/clustering/data
data_dir:  /home/lyuzhuoqi/projects/clustering/data
data_dir:  /home/lyuzhuoqi/projects/clustering/data


Processing JSON: 100%|████████████████████████████████████████████████████▉| 177G/177G [09:05<00:00, 324MB/s]

成功提取 23322430 篇论文摘要。
成功提取 23322430 篇论文摘要


In [7]:
import json

def read_lines(file_path, n: int=2):
    """
    从 Flat JSON 文件中读取前两行并打印解析结果。
    
    Args:
        file_path (str): Flat JSON 文件路径（每行一个 JSON 对象）。
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        for idx, line in enumerate(f):
            if idx < n:  # 只处理前n行
                try:
                    parsed_line = json.loads(line.strip())  # 解析 JSON 对象
                    print(json.dumps(parsed_line, indent=4, ensure_ascii=False))  # 美化输出
                except json.JSONDecodeError as e:
                    print(f"解析失败：{e}, 原始数据：{line.strip()}")
            else:
                break

In [8]:
output_path = os.path.join(get_data_dir(), "2010s", "dataset.json")
read_lines(output_path, 20)

data_dir:  /home/lyuzhuoqi/projects/clustering/data
{
    "PaperID": 2055207701,
    "abstract": "Abstract Lake Trout Salvelinus namaycush were extirpated from Lake Michigan by the early 1950s, and as part of an effort to restore naturally reproducing populations, hatchery-reared fish have been stocked since the early 1960s. Stocked fish are marked with a fin clip to differentiate them from wild, lake-produced Lake Trout; marking error for the 2007–2010 year-classes of Lake Trout stocked by federal hatcheries averaged 3.0%. Egg deposition, emergent fry, and wild juvenile Lake Trout have previously been observed, but no sustained wild recruitment has been measured in assessment surveys or in sport and commercial fishery catches. In 2011 and 2012, we caught juvenile Lake Trout in gill-net and bottom trawl catches that were targeting Bloater Coregonus hoyi in water depths greater than 80 m. Unclipped, wild Lake Trout represented 20% of all Lake Trout caught in a southern offshore region o